In [ ]:
import sys
import json
import os
import time
from google.cloud import bigquery

sys.path.append('kafka_workspace')
import kafka_util
import bq_util

with open('kafka_workspace/sample_message.json') as file:
    mesg = json.loads(file.read())

project_id = 'pensjon-saksbehandli-dev-cb76'
dataset_id = 'pensjon_malinger'
table_name = 'test_kafka_oppgave'
bq_client = bigquery.Client(project=project_id)
table_id = bq_util.create_bq_table_if_not_exists(bq_client, project_id, dataset_id, table_name)

print(mesg)

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Dict, Any


@dataclass
class Message:
    hendelsestype: str
    tidspunkt: 'datetime'
    tema: str
    oppgavetype: str
    aktiv_fra: 'datetime'
    aktiv_til: 'datetime'
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        return Message(hendelsestype=data["hendelse"]["hendelsestype"],
                       tidspunkt=data["hendelse"]["tidspunkt"],
                       tema=data["oppgave"]["kategorisering"]["tema"],
                       oppgavetype=data["oppgave"]["kategorisering"]["oppgavetype"],
                       aktiv_fra=datetime.fromisoformat(data["oppgave"]["behandlingsperiode"]["aktiv"]).timestamp(),
                       aktiv_til=datetime.fromisoformat(data["oppgave"]["behandlingsperiode"]["frist"]).timestamp(),
        )
    
    def to_dict(self):
        _dict = self.__dict__
        return {key: value for key, value in _dict.items()}

In [ ]:
message = Message.from_dict(mesg).to_dict()

In [ ]:
errors = bq_client.insert_rows_json(table_id, [message])

In [ ]:
errors